In [3]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my-test-app")
location = geolocator.geocode("千葉県 千葉市中央区矢作町")

print("住所:", location.address)
print("緯度:", location.latitude)
print("経度:", location.longitude)

住所: 千葉県立中央博物館, 千葉市中央区青葉町９５５－２, 公園北通り, 青葉町, 中央区, 千葉市, 千葉県, 260-0853, 日本
緯度: 35.599867
経度: 140.138489


In [6]:
import requests
import urllib
makeUrl = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="
s_quote = urllib.parse.quote('千葉県 千葉市中央区末広4丁目6番9')
response = requests.get(makeUrl + s_quote)
print(response.json() )

[{'geometry': {'coordinates': [140.127182, 35.591328], 'type': 'Point'}, 'type': 'Feature', 'properties': {'addressCode': '', 'title': '千葉県千葉市中央区末広四丁目６番９号'}}]


In [13]:
import pandas as pd
import requests
import urllib
import folium

# CSV読み込み（Shift-JIS/CP932対応）
df = pd.read_csv('chibalandprice.csv', dtype=str, encoding='cp932')

# 緯度経度カラム作成
df['lat'] = None
df['lon'] = None

# 国土地理院API
base_url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

for i, address in df['reallocation'].items():
    try:
        s_quote = urllib.parse.quote(address)
        response = requests.get(base_url + s_quote)
        data = response.json()
        if data:
            df.at[i, 'lat'] = float(data[0]['geometry']['coordinates'][1])
            df.at[i, 'lon'] = float(data[0]['geometry']['coordinates'][0])
    except Exception as e:
        print(f"Error at row {i}: {e}")

# NaN除去
df = df.dropna(subset=['lat', 'lon'])

# 地図作成
map_center = [df['lat'].astype(float).mean(), df['lon'].astype(float).mean()]
m = folium.Map(location=map_center, zoom_start=14)

for _, row in df.iterrows():
    folium.CircleMarker(
        location=[float(row['lat']), float(row['lon'])],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        popup=f"地価: {row['postedLandPriceOfR05']}"
    ).add_to(m)

m.save('land_price_map.html')
print("地図を land_price_map.html に保存しました。")

地図を land_price_map.html に保存しました。


In [18]:
import pandas as pd
import requests, urllib
import folium
from branca.colormap import linear, LinearColormap

# ==== CSV読み込み（文字コード対策） ====
try:
    df = pd.read_csv('chibalandprice.csv', dtype=str, encoding='cp932')
except UnicodeDecodeError:
    df = pd.read_csv('chibalandprice.csv', dtype=str, encoding='utf-8')

# ==== 住所→緯度経度（既に lat/lon があるならこのブロックはスキップ可） ====
if 'lat' not in df.columns or 'lon' not in df.columns:
    df['lat'] = None
    df['lon'] = None
base_url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="
for i, address in df['reallocation'].items():
    if pd.notna(df.at[i,'lat']) and pd.notna(df.at[i,'lon']):
        continue
    try:
        s_quote = urllib.parse.quote(address)
        data = requests.get(base_url + s_quote, timeout=10).json()
        if data:
            df.at[i, 'lat'] = float(data[0]['geometry']['coordinates'][1])
            df.at[i, 'lon'] = float(data[0]['geometry']['coordinates'][0])
    except Exception as e:
        print(f"[warn] row {i}: {e}")

# 緯度経度が無い行は除外 & 数値化
df = df.dropna(subset=['lat','lon']).copy()
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)
df['postedLandPriceOfR05'] = pd.to_numeric(df['postedLandPriceOfR05'], errors='coerce')
df = df.dropna(subset=['postedLandPriceOfR05'])

# ==== 地価レンジ ====
min_price = df['postedLandPriceOfR05'].min()
max_price = df['postedLandPriceOfR05'].max()
rng = (max_price - min_price) if max_price > min_price else 1.0  # 0割防止

# ==== カラーマップ（順序は reversed → scale） ====
try:
    # これが通れば一番簡単（青→黄→赤）
    colormap = linear.RdYlBu_11.reversed().scale(min_price, max_price)
except Exception:
    try:
        # 古い版でも動くことが多い逆順プリセット
        colormap = linear.RdYlBu_11_r.scale(min_price, max_price)
    except Exception:
        # 最終手段：手動で近い配色を指定（青→…→赤）
        colormap = LinearColormap(
            colors=['#313695','#4575b4','#74add1','#abd9e9','#e0f3f8',
                    '#ffffbf','#fee090','#fdae61','#f46d43','#d73027'],
            vmin=min_price, vmax=max_price
        )
colormap.caption = '地価 (R05年度)'

# ==== 地図 ====
m = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=13)

# ヒートマップ風：色＋サイズを地価で連続変化
for _, r in df.iterrows():
    price = r['postedLandPriceOfR05']
    ratio = (price - min_price) / rng
    radius = 3 + ratio * 15  # 3〜18px
    color = colormap(price)
    folium.CircleMarker(
        location=[r['lat'], r['lon']],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"地価: {price:,.0f}"
    ).add_to(m)

colormap.add_to(m)
m.save('land_price_heatmap_full.html')
print("land_price_heatmap_full.html を出力しました。")

land_price_heatmap_full.html を出力しました。
